#NAMA : YUNAENA MARATUL KIROM

#NIM : 121450080

#KELAS : TBD RC

##A Dataset to Play With

In [ ]:
import numpy as np
import pickle
from pathlib import Path

# Path to the unzipped CIFAR data
data_dir = Path("data/cifar-10-batches-py/")

# Unpickle function provided by the CIFAR hosts
def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

images, labels = [], []
for batch in data_dir.glob("data_batch_*"):
    batch_data = unpickle(batch)
    for i, flat_im in enumerate(batch_data[b"data"]):
        im_channels = []
        # Each image is flattened, with channels in order of R, G, B
        for j in range(3):
            im_channels.append(
                flat_im[j * 1024 : (j + 1) * 1024].reshape((32, 32))
            )
        # Reconstruct the original image
        images.append(np.dstack((im_channels)))
        # Save the label
        labels.append(batch_data[b"labels"][i])

print("Loaded CIFAR-10 training set:")
print(f" - np.shape(images)     {np.shape(images)}")
print(f" - np.shape(labels)     {np.shape(labels)}")

Loaded CIFAR-10 training set:
 - np.shape(images)     (0,)
 - np.shape(labels)     (0,)


Analisis : Kode di atas adalah sebuah skrip Python yang bertujuan untuk memuat dataset CIFAR-10, yang merupakan kumpulan data gambar. serta merepresentasikan data tersebut dalam format yang sesuai untuk dilakukan proses pelatihan atau pengujian algoritma pembelajaran mesin. Pertama, diinisialisasi beberapa modul yang diperlukan, yaitu numpy untuk manipulasi data numerik, pickle untuk membaca dan menulis objek Python ke file, dan Path dari pathlib untuk memanipulasi path file dan direktori. Dilanjutkan dengan mendefinisikan path ke direktori yang berisi data CIFAR-10 yang telah di-unzip. Ini dilakukan menggunakan Path dari modul pathlib. Kemudian, ada sebuah fungsi bernama unpickle yang didefinisikan. Fungsi ini bertugas untuk membaca file yang sudah di-"unpickle" dan mengembalikan kamus (dictionary) yang berisi data.
Setelah itu, dilakukan iterasi melalui setiap batch data dalam direktori data CIFAR menggunakan loop for. Setiap batch di-"unpickled" menggunakan fungsi unpickle yang telah didefinisikan sebelumnya.
Selanjutnya, untuk setiap gambar dalam batch, iterasi dilakukan dengan mengambil gambar yang sudah di-"flattened" (diratakan) dalam bentuk array 1D. Kemudian, array ini dipecah menjadi tiga bagian sesuai dengan channel warna (R, G, B) masing-masing gambar.
Dengan channel-channel tersebut, gambar yang asli direkonstruksi dengan menggunakan np.dstack untuk membuat stack array dengan tiga dimensi, merepresentasikan gambar berwarna.
Selama iterasi, label untuk setiap gambar juga disimpan.
Setelah semua batch diproses, cetaklah informasi tentang dataset CIFAR-10 yang telah dimuat, yaitu bentuk (shape) dari array gambar (images) dan array label (labels).

##Setup for Storing Images on Disk

In [ ]:
$ pip install Pillow

In [ ]:
$ conda install -c conda-forge pillow

##Getting Started With LMDB

In [ ]:
$ pip install lmdb

In [ ]:
$ conda install -c conda-forge python-lmdb

Analisis : Kode di atas menunjukkan dua perintah yang digunakan untuk menginstal dua paket Python yang diperlukan untuk menjalankan kode tertentu. Perintah pip install Pillow digunakan untuk menginstal paket Pillow. Pillow adalah sebuah library Python yang berguna untuk manipulasi gambar. Perintah conda install -c conda-forge python-lmdb digunakan untuk menginstal paket python-lmdb menggunakan Anaconda. Paket ini memungkinkan akses ke basis data berorientasi kunci yang cepat dengan menggunakan penyimpanan berbasis file.

##Getting Started With HDF5

In [ ]:
$ pip install h5py

In [ ]:
$ conda install -c conda-forge h5py

##Storing a Single Image

In [ ]:
from pathlib import Path

disk_dir = Path("data/disk/")
lmdb_dir = Path("data/lmdb/")
hdf5_dir = Path("data/hdf5/")

In [ ]:
disk_dir.mkdir(parents=True, exist_ok=True)
lmdb_dir.mkdir(parents=True, exist_ok=True)
hdf5_dir.mkdir(parents=True, exist_ok=True)

Analisis : kode tersebut memastikan bahwa ketiga direktori yang diperlukan untuk menyimpan data dalam format disk, LMDB, dan HDF5 tersedia, dan siap digunakan dalam proses selanjutnya. Pertama-tama, direktori data/disk/ dibuat menggunakan metode mkdir dari objek Path. Parameter parents=True digunakan untuk membuat direktori induk secara rekursif jika diperlukan, dan exist_ok=True digunakan untuk mengabaikan kesalahan jika direktori tersebut sudah ada.
Langkah yang sama dilakukan untuk membuat direktori data/lmdb/ dan data/hdf5/ dengan menggunakan objek Path yang sesuai untuk masing-masing direktori.

##Storing to Disk

In [ ]:
from PIL import Image
import csv

def store_single_disk(image, image_id, label):
    """ Stores a single image as a .png file on disk.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    Image.fromarray(image).save(disk_dir / f"{image_id}.png")

    with open(disk_dir / f"{image_id}.csv", "wt") as csvfile:
        writer = csv.writer(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        writer.writerow([label])

##Storing to LMDB

In [ ]:
class CIFAR_Image:
    def __init__(self, image, label):
        # Dimensions of image for reconstruction - not really necessary
        # for this dataset, but some datasets may include images of
        # varying sizes
        self.channels = image.shape[2]
        self.size = image.shape[:2]

        self.image = image.tobytes()
        self.label = label

    def get_image(self):
        """ Returns the image as a numpy array. """
        image = np.frombuffer(self.image, dtype=np.uint8)
        return image.reshape(*self.size, self.channels)

In [ ]:
import lmdb
import pickle

def store_single_lmdb(image, image_id, label):
    """ Stores a single image to a LMDB.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    map_size = image.nbytes * 10

    # Create a new LMDB environment
    env = lmdb.open(str(lmdb_dir / f"single_lmdb"), map_size=map_size)

    # Start a new write transaction
    with env.begin(write=True) as txn:
        # All key-value pairs need to be strings
        value = CIFAR_Image(image, label)
        key = f"{image_id:08}"
        txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

Analisis : Kelas CIFAR_Image dan fungsi store_single_lmdb bekerja sama untuk menyimpan gambar dan labelnya ke dalam basis data berorientasi kunci (key-value) LMDB.

##Storing With HDF5

In [ ]:
import h5py

def store_single_hdf5(image, image_id, label):
    """ Stores a single image to an HDF5 file.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{image_id}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "image", np.shape(image), h5py.h5t.STD_U8BE, data=image
    )
    meta_set = file.create_dataset(
        "meta", np.shape(label), h5py.h5t.STD_U8BE, data=label
    )
    file.close()

Analisis : Fungsi store_single_hdf5 digunakan untuk menyimpan satu gambar ke dalam sebuah file HDF5, bersama dengan labelnya.

##Experiments for Storing a Single Image

In [ ]:
_store_single_funcs = dict(
    disk=store_single_disk, lmdb=store_single_lmdb, hdf5=store_single_hdf5
)

Analisis : Dictionary _store_single_funcs didefinisikan untuk menyimpan fungsi-fungsi yang bertujuan untuk menyimpan data gambar ke dalam berbagai format penyimpanan, yaitu disk, LMDB, dan HDF5. Setiap format penyimpanan memiliki sebuah kunci yang terkait dengan fungsi yang sesuai untuk menyimpan data ke dalam format tersebut.

In [ ]:
from timeit import timeit

store_single_timings = dict()

for method in ("disk", "lmdb", "hdf5"):
    t = timeit(
        "_store_single_funcs[method](image, 0, label)",
        setup="image=images[0]; label=labels[0]",
        number=1,
        globals=globals(),
    )
    store_single_timings[method] = t
    print(f"Method: {method}, Time usage: {t}")

Analisis : Kode di atas mengukur waktu yang diperlukan untuk menyimpan sebuah gambar dan labelnya menggunakan tiga metode penyimpanan yang berbeda: disk, LMDB, dan HDF5. Modul timeit diimpor dari timeit untuk mengukur waktu eksekusi fungsi.
Sebuah dictionary bernama store_single_timings didefinisikan untuk menyimpan waktu eksekusi (dalam detik) untuk setiap metode penyimpanan.
Dilakukan iterasi melalui tiga metode penyimpanan yang berbeda, yaitu "disk", "lmdb", dan "hdf5".
Pada setiap iterasi, fungsi timeit() digunakan untuk mengukur waktu eksekusi dari fungsi penyimpanan yang sesuai untuk metode tersebut.

##Storing Many Images

In [ ]:
store_many_disk(images, labels):
    """ Stores an array of images to disk
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Save all the images one by one
    for i, image in enumerate(images):
        Image.fromarray(image).save(disk_dir / f"{i}.png")

    # Save all the labels to the csv file
    with open(disk_dir / f"{num_images}.csv", "w") as csvfile:
        writer = csv.writer(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for label in labels:
            # This typically would be more than just one value per row
            writer.writerow([label])

def store_many_lmdb(images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    env = lmdb.open(str(lmdb_dir / f"{num_images}_lmdb"), map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = CIFAR_Image(images[i], labels[i])
            key = f"{i:08}"
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

def store_many_hdf5(images, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_many.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()

Analisis : Fungsi store_many_disk, store_many_lmdb, dan store_many_hdf5 adalah implementasi dari fungsi-fungsi yang bertujuan untuk menyimpan sejumlah gambar beserta labelnya ke dalam disk, LMDB, dan HDF5 secara berurutan.

##Preparing the Dataset

In [ ]:
cutoffs = [10, 100, 1000, 10000, 100000]

# Let's double our images so that we have 100,000
images = np.concatenate((images, images), axis=0)
labels = np.concatenate((labels, labels), axis=0)

# Make sure you actually have 100,000 images and labels
print(np.shape(images))
print(np.shape(labels))

Analisis : kode di atas, dilakukan penambahan jumlah gambar dan label menjadi 100.000 dengan menggandakan dataset yang ada. Kemudian, dilakukan pencetakan untuk memastikan bahwa jumlah gambar dan label benar-benar telah menjadi 100.000. List cutoffs didefinisikan dengan nilai-nilai [10, 100, 1000, 10000, 100000]. List ini mungkin digunakan sebagai nilai batas atau ambang yang akan digunakan dalam analisis atau pemrosesan data selanjutnya.
Jumlah gambar dan label digandakan menjadi 100.000 dengan menggunakan fungsi np.concatenate() untuk menggabungkan dataset yang ada dengan dirinya sendiri pada sumbu pertama (axis=0). Hal ini dilakukan agar jumlah gambar dan label benar-benar menjadi 100.000.
Dilakukan pencetakan untuk memastikan bahwa jumlah gambar dan label telah menjadi 100.000 dengan menggunakan np.shape() untuk mendapatkan bentuk dari array images dan labels, dan kemudian mencetaknya.

##Experiment for Storing Many Images

In [ ]:
_store_many_funcs = dict(
    disk=store_many_disk, lmdb=store_many_lmdb, hdf5=store_many_hdf5
)

from timeit import timeit

store_many_timings = {"disk": [], "lmdb": [], "hdf5": []}

for cutoff in cutoffs:
    for method in ("disk", "lmdb", "hdf5"):
        t = timeit(
            "_store_many_funcs[method](images_, labels_)",
            setup="images_=images[:cutoff]; labels_=labels[:cutoff]",
            number=1,
            globals=globals(),
        )
        store_many_timings[method].append(t)

        # Print out the method, cutoff, and elapsed time
        print(f"Method: {method}, Time usage: {t}")

Analisis : kode di atas, dilakukan pengukuran waktu eksekusi untuk menyimpan sejumlah gambar dan label dalam tiga metode penyimpanan yang berbeda (disk, LMDB, dan HDF5) dengan menggunakan berbagai nilai ambang (cutoffs). Dictionary _store_many_funcs didefinisikan untuk menyimpan fungsi-fungsi yang bertujuan untuk menyimpan sejumlah gambar dan label dalam format penyimpanan yang berbeda: disk, LMDB, dan HDF5. Setiap format penyimpanan memiliki sebuah kunci yang terkait dengan fungsi yang sesuai untuk menyimpan data dalam format tersebut.
Dictionary store_many_timings didefinisikan untuk menyimpan waktu eksekusi (dalam detik) untuk setiap metode penyimpanan, di mana kunci dictionary adalah nama metode penyimpanan (disk, lmdb, hdf5).
Dilakukan perulangan melalui setiap nilai ambang dalam list cutoffs untuk menyimpan sejumlah gambar dan label yang berbeda.
Pada setiap iterasi, dilakukan perulangan melalui tiga metode penyimpanan yang berbeda: disk, LMDB, dan HDF5.
Fungsi timeit() digunakan untuk mengukur waktu eksekusi dari fungsi penyimpanan yang sesuai untuk metode tersebut.

In [ ]:
import matplotlib.pyplot as plt

def plot_with_legend(
    x_range, y_data, legend_labels, x_label, y_label, title, log=False
):
    """ Displays a single plot with multiple datasets and matching legends.
        Parameters:
        --------------
        x_range         list of lists containing x data
        y_data          list of lists containing y values
        legend_labels   list of string legend labels
        x_label         x axis label
        y_label         y axis label
    """
    plt.style.use("seaborn-whitegrid")
    plt.figure(figsize=(10, 7))

    if len(y_data) != len(legend_labels):
        raise TypeError(
            "Error: number of data sets does not match number of labels."
        )

    all_plots = []
    for data, label in zip(y_data, legend_labels):
        if log:
            temp, = plt.loglog(x_range, data, label=label)
        else:
            temp, = plt.plot(x_range, data, label=label)
        all_plots.append(temp)

    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(handles=all_plots)
    plt.show()

# Getting the store timings data to display
disk_x = store_many_timings["disk"]
lmdb_x = store_many_timings["lmdb"]
hdf5_x = store_many_timings["hdf5"]

plot_with_legend(
    cutoffs,
    [disk_x, lmdb_x, hdf5_x],
    ["PNG files", "LMDB", "HDF5"],
    "Number of images",
    "Seconds to store",
    "Storage time",
    log=False,
)

plot_with_legend(
    cutoffs,
    [disk_x, lmdb_x, hdf5_x],
    ["PNG files", "LMDB", "HDF5"],
    "Number of images",
    "Seconds to store",
    "Log storage time",
    log=True,
)

Analisis : Fungsi plot_with_legend digunakan untuk membuat plot yang menampilkan beberapa dataset dengan label-legenda yang sesuai. Fungsi menerima beberapa parameter:
x_range: List dari list yang berisi data x.
y_data: List dari list yang berisi data y.
legend_labels: List dari string yang berisi label-legenda untuk setiap dataset.
x_label: Label sumbu x.
y_label: Label sumbu y.
title: Judul plot.
log: Argumen opsional untuk menentukan apakah plot akan menggunakan skala logaritmik atau tidak. Default-nya adalah False.
Terlebih dahulu, gaya plot menggunakan plt.style.use("seaborn-whitegrid") dan ukuran plot diatur menjadi figsize=(10, 7).
Dilakukan pengecekan untuk memastikan bahwa jumlah dataset (y_data) sama dengan jumlah label-legenda (legend_labels). Jika tidak, akan muncul pesan kesalahan.
Selanjutnya, dilakukan iterasi melalui y_data dan legend_labels. Untuk setiap pasangan data dan label-legenda, plot dibuat menggunakan plt.plot() atau plt.loglog() tergantung pada nilai parameter log. Hasil dari fungsi ini disimpan dalam variabel temp dan ditambahkan ke dalam list all_plots.
Setelah selesai melakukan iterasi, judul plot, label sumbu x, dan label sumbu y ditambahkan dengan menggunakan plt.title(), plt.xlabel(), dan plt.ylabel().
Legenda ditambahkan ke plot menggunakan plt.legend(), dengan menggunakan list all_plots sebagai inputnya.
Plot ditampilkan dengan menggunakan plt.show().

##Reading a Single Image

Reading From Disk

In [ ]:
def read_single_disk(image_id):
    """ Stores a single image to disk.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    image = np.array(Image.open(disk_dir / f"{image_id}.png"))

    with open(disk_dir / f"{image_id}.csv", "r") as csvfile:
        reader = csv.reader(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        label = int(next(reader)[0])

    return image, label

Reading From LMDB

In [ ]:
def read_single_lmdb(image_id):
    """ Stores a single image to LMDB.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    # Open the LMDB environment
    env = lmdb.open(str(lmdb_dir / f"single_lmdb"), readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Encode the key the same way as we stored it
        data = txn.get(f"{image_id:08}".encode("ascii"))
        # Remember it's a CIFAR_Image object that is loaded
        cifar_image = pickle.loads(data)
        # Retrieve the relevant bits
        image = cifar_image.get_image()
        label = cifar_image.label
    env.close()

    return image, label

Analisis : Fungsi read_single_lmdb digunakan untuk membaca satu gambar dari basis data LMDB. ungsi menerima satu parameter:
image_id: ID unik dalam bentuk integer untuk gambar yang akan dibaca.
Lingkungan (environment) LMDB dibuka dalam mode hanya baca (readonly) menggunakan lmdb.open().
Transaksi baca baru dimulai menggunakan env.begin(). Transaksi ini memungkinkan pembacaan data dari basis data.
Kunci (key) yang sesuai dengan image_id dienkripsi menggunakan format yang sama seperti saat data disimpan dengan menggunakan f"{image_id:08}".encode("ascii").
Data yang sesuai dengan kunci tersebut dibaca dari basis data menggunakan txn.get(). Data yang dibaca adalah objek CIFAR_Image yang sebelumnya disimpan.
Objek CIFAR_Image di-deserialize kembali menggunakan pickle.loads().
Gambar dan label yang sesuai dengan objek CIFAR_Image dibaca dan diambil.
Setelah selesai membaca data, lingkungan LMDB ditutup dengan menggunakan env.close().
Gambar dan label yang dibaca kemudian dikembalikan sebagai output dari fungsi.

Reading From HDF5

In [ ]:
def read_single_hdf5(image_id):
    """ Stores a single image to HDF5.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{image_id}.h5", "r+")

    image = np.array(file["/image"]).astype("uint8")
    label = int(np.array(file["/meta"]).astype("uint8"))

    return image, label

Analisis : Fungsi read_single_hdf5 bertujuan untuk membaca satu gambar dan label yang terkait dari file HDF5.

In [ ]:
_read_single_funcs = dict(
    disk=read_single_disk, lmdb=read_single_lmdb, hdf5=read_single_hdf5
)

Experiment for Reading a Single Image

In [ ]:
from timeit import timeit

read_single_timings = dict()

for method in ("disk", "lmdb", "hdf5"):
    t = timeit(
        "_read_single_funcs[method](0)",
        setup="image=images[0]; label=labels[0]",
        number=1,
        globals=globals(),
    )
    read_single_timings[method] = t
    print(f"Method: {method}, Time usage: {t}")

Analisis : kode di atas mengukur waktu yang diperlukan untuk membaca satu gambar dan label terkait dari tiga metode penyimpanan yang berbeda (disk, LMDB, dan HDF5).

##Reading Many Images

Adjusting the Code for Many Images

In [ ]:
def read_many_disk(num_images):
    """ Reads image from disk.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Loop over all IDs and read each image in one by one
    for image_id in range(num_images):
        images.append(np.array(Image.open(disk_dir / f"{image_id}.png")))

    with open(disk_dir / f"{num_images}.csv", "r") as csvfile:
        reader = csv.reader(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for row in reader:
            labels.append(int(row[0]))
    return images, labels

def read_many_lmdb(num_images):
    """ Reads image from LMDB.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []
    env = lmdb.open(str(lmdb_dir / f"{num_images}_lmdb"), readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object
            # that is stored as the value
            cifar_image = pickle.loads(data)
            # Retrieve the relevant bits
            images.append(cifar_image.get_image())
            labels.append(cifar_image.label)
    env.close()
    return images, labels

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_many.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/meta"]).astype("uint8")

    return images, labels

_read_many_funcs = dict(
    disk=read_many_disk, lmdb=read_many_lmdb, hdf5=read_many_hdf5
)

Analisis : Fungsi read_many_disk, read_many_lmdb, dan read_many_hdf5 bertujuan untuk membaca sejumlah gambar dan label yang terkait dari tiga metode penyimpanan yang berbeda: disk, LMDB, dan HDF5. Fungsi read_many_disk:

Fungsi menerima satu parameter:
num_images: Jumlah gambar yang akan dibaca.
Array kosong images dan labels dideklarasikan untuk menampung gambar dan label yang akan dibaca.
Dilakukan perulangan dari 0 hingga num_images - 1 untuk membaca setiap gambar secara berurutan dari disk.
Setiap gambar dibuka menggunakan Image.open() dari modul PIL dan diubah menjadi array NumPy menggunakan np.array().
Array gambar kemudian ditambahkan ke dalam list images.
Label-label yang terkait dengan gambar-gambar tersebut dibaca dari file CSV terpisah dan disimpan ke dalam list labels.
Setelah selesai membaca semua gambar dan label, keduanya dikembalikan sebagai output dari fungsi.

Fungsi read_many_lmdb:

Fungsi menerima satu parameter yang sama dengan fungsi read_many_disk.
Array kosong images dan labels dideklarasikan untuk menampung gambar dan label yang akan dibaca.
Lingkungan LMDB dibuka dalam mode hanya baca (readonly) menggunakan lmdb.open().
Dilakukan perulangan dari 0 hingga num_images - 1 untuk membaca setiap gambar secara berurutan dari basis data LMDB.
Setiap gambar dan label yang terkait dibaca menggunakan transaksi baca LMDB yang sesuai.
Array gambar dan label kemudian ditambahkan ke dalam list images dan labels.
Setelah selesai membaca semua gambar dan label, keduanya dikembalikan sebagai output dari fungsi.

Fungsi read_many_hdf5:

Fungsi menerima satu parameter yang sama dengan fungsi read_many_disk.
Array kosong images dan labels dideklarasikan untuk menampung gambar dan label yang akan dibaca.
File HDF5 dibuka menggunakan h5py.File() dalam mode baca-tulis ("r+").
Data gambar dan label dibaca dari dataset yang sesuai di dalam file HDF5 menggunakan np.array().
Data gambar dibaca dari dataset "images" dan disimpan dalam array images.
Data label dibaca dari dataset "meta" dan disimpan dalam array labels.
Setelah selesai membaca semua gambar dan label, keduanya dikembalikan sebagai output dari fungsi.

Experiment for Reading Many Images

In [ ]:
from timeit import timeit

read_many_timings = {"disk": [], "lmdb": [], "hdf5": []}

for cutoff in cutoffs:
    for method in ("disk", "lmdb", "hdf5"):
        t = timeit(
            "_read_many_funcs[method](num_images)",
            setup="num_images=cutoff",
            number=1,
            globals=globals(),
        )
        read_many_timings[method].append(t)

        # Print out the method, cutoff, and elapsed time
        print(f"Method: {method}, No. images: {cutoff}, Time usage: {t}")

Analisis : kode di atas digunakan untuk mengukur waktu yang diperlukan untuk membaca sejumlah gambar dan label yang berbeda dari tiga metode penyimpanan yang berbeda (disk, LMDB, dan HDF5) dengan menggunakan berbagai jumlah gambar (cutoffs).

A More Critical Look at Implementation

In [ ]:
# Slightly slower
for i in range(len(dataset)):
    # Read the ith value in the dataset, one at a time
    do_something_with(dataset[i])

# This is better
data = dataset[:]
for d in data:
    do_something_with(d)

Analisis : kode di atas memberikan dua pendekatan yang berbeda dalam melakukan iterasi melalui dataset.